In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

from tqdm import tqdm
import matplotlib.pyplot as plt

from model import HopfieldNetwork
from utils.dataset import PreloadedDataset
from utils.train import train_denoise

c:\Users\Joe\anaconda3\envs\ml-env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
VAL_RATIO = 0.2

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
    # transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.9, 1.1))
])

dataset = datasets.MNIST(root="datasets/", train=True, transform=transforms.ToTensor(), download=True)
val_len = int(len(dataset) * VAL_RATIO)
train_len = len(dataset) - val_len
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_len, val_len])
train_dataset = PreloadedDataset.from_dataset(train_dataset, transform, device)
val_dataset = PreloadedDataset.from_dataset(val_dataset, transforms.ToTensor(), device)
INPUT_SHAPE = train_dataset[0][0].shape
NUM_CLASSES = 10

In [6]:
# Hebbian Learning
torch.manual_seed(42)
model_name = "mnist_hopfield_hebbian"
size = 1
for dim in INPUT_SHAPE:
    size *= dim
model = HopfieldNetwork(size, bias=True, steps=10).to(device)
step = 0

In [7]:
LR = 3e-8
BATCH_SIZE=64
criterion = nn.MSELoss()
optimiser = optim.SGD(model.parameters(), lr=LR)
step = train_denoise(model, train_dataset, val_dataset, optimiser, criterion, model_name, 10, minimise='loss', batch_size=BATCH_SIZE, flatten=True, step=step, device=device)

KeyboardInterrupt: 